In [1]:
from rocketshp import config

2024-09-17 14:55:38.912 | INFO     | rocketshp.config:<module>:11 - PROJ_ROOT path is: /mnt/home/ssledzieski/Projects/rocketshp


In [2]:
from loguru import logger

In [3]:
import h5py
from torchmdnet.datasets import MDCATH

In [4]:
MDCATH_DIR = f"{config.RAW_DATA_DIR}/mdcath"
TEMP = "348"
REPL = "0"

In [5]:
logger.info('Loading MDCath')
mdc = MDCATH(MDCATH_DIR)

2024-09-17 14:55:46.390 | INFO     | __main__:<module>:1 - Loading MDCath


Processing mdcath source: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5398/5398 [00:05<00:00, 933.18it/s]


In [6]:
from rocketshp.convert_mdCATH import convert_to_files, convert_to_mdtraj

In [7]:
DOMAIN = "2nttA02"

In [64]:
pdb_fi, xtc_fi = convert_to_files(f"{MDCATH_DIR}/mdcath_dataset_{DOMAIN}.h5", f"{config.INTERIM_DATA_DIR}/{DOMAIN}" , ["348"], ["3"])

2024-09-17 11:02:31.398 | INFO     | rocketshp.convert_mdCATH:convert_to_files:176 - Wrote /mnt/home/ssledzieski/Projects/rocketshp/data/interim/2nttA02.pdb
2024-09-17 11:02:31.627 | INFO     | rocketshp.convert_mdCATH:convert_to_files:183 - Wrote /mnt/home/ssledzieski/Projects/rocketshp/data/interim/2nttA02_348_3.xtc


In [65]:
!ls

 0.01_explore_mdcath.ipynb  'View Flexibility.ipynb'


In [67]:
import mdtraj

In [77]:
traj = convert_to_mdtraj(f"{MDCATH_DIR}/mdcath_dataset_{DOMAIN}.h5", TEMP, REPL)
traj_sp = traj.superpose(traj[0])

In [78]:
traj_sp

<mdtraj.Trajectory with 500 frames, 1099 atoms, 70 residues, without unitcells at 0x7f3936d16ed0>

In [79]:
import nglview as nv

In [80]:
nv.show_pdbid(DOMAIN[:4])

NGLWidget()

In [82]:
view = nv.show_mdtraj(traj_sp)
view.remove_cartoon()
view.add_cartoon('protein')
view

NGLWidget(max_frame=499)

In [71]:
pm = mdtraj.principal_moments(traj_sp)

In [72]:
pm.shape

(500, 3)

In [73]:
pm

array([[0.27976751, 0.4409481 , 0.60691768],
       [0.27910251, 0.44235817, 0.6225342 ],
       [0.29659791, 0.40083691, 0.65179232],
       ...,
       [0.29595708, 0.39087462, 0.99456048],
       [0.2927136 , 0.4095155 , 0.95013262],
       [0.2843157 , 0.42080685, 0.9946145 ]])

In [18]:
def get_mdcath_coords(mdc_object, domain, frame, temp=TEMP, repl=REPL):
    z, coords, forces = mdc_object.process_specific_group(domain, f"{MDCATH_DIR}/mdcath_dataset_{domain}.h5",temp, repl, frame)
    return coords

In [40]:
def get_mdcath_entry(domain, temp=TEMP, repl=REPL):
    with h5py.File(f"{MDCATH_DIR}/mdcath_dataset_{domain}.h5","r") as fi:
        residue_name = fi[f"{domain}/resname"][:]
        residue_id = fi[f"{domain}/resid"][:]
        atom_list = fi[f"{domain}/element"][:]
        sim = fi[f"{domain}/{temp}/{repl}"]
        coords = sim["coords"][:]
        gyration_radius = sim["gyrationRadius"][:]
    return residue_name, residue_id, atom_list, coords, gyration_radius

In [48]:
residues, resid, atoms, coords, gr = get_mdcath_entry("2nttA02")

In [52]:
seq = "MEMEKEFEQIDKSGSWAAIYQDIRHEASDFPCRVAKLPKNKNRNRYRDVSPFDHSRIKLHQEDNDYINASLIKMEEAQRSYILTQGPLPNTCGHFWEMVWEQKSRGVVMLNRVMEKGSLKCAQYWPQKEEKEMIFEDTNLKLTLISEDIKSYYTVRQLELENLTTQETREILHFHYTTWPDFGVPESPASFLNFLFKVRESGSLSPEHGPVVVHCSAGIGRSGTFCLADTCLLLMDKRKDPSSVDIKKVLLDMRKFRMGLIQTADQLRFSYLAVIEGAKFIMGDSSVQDQWKELSHEDL"

In [55]:
seq[10:30]

'DKSGSWAAIYQDIRHEASDF'

In [49]:
residues[:50]

array([b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'VAL', b'VAL',
       b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL',
       b'VAL', b'VAL'], dtype=object)

In [50]:
resid[:50]

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19])

In [51]:
atoms[:50]

array([b'C', b'H', b'H', b'H', b'C', b'O', b'N', b'H', b'C', b'H', b'C',
       b'H', b'H', b'C', b'O', b'O', b'C', b'O', b'N', b'H', b'C', b'H',
       b'C', b'H', b'H', b'C', b'C', b'H', b'C', b'H', b'C', b'H', b'C',
       b'H', b'C', b'H', b'C', b'O', b'N', b'H', b'C', b'H', b'C', b'H',
       b'C', b'H', b'H', b'H', b'C', b'H'], dtype=object)

In [21]:
get_mdcath_coords(mdc, "153lA00", 1)

array([[ 2.82000017, 49.13999939, 75.39000702],
       [ 2.9000001 , 48.18000412, 75.13000488],
       [ 3.80000019, 49.72999954, 76.19000244],
       ...,
       [ 2.6900003 , 40.33000183, 60.29000092],
       [ 0.67000002, 43.22000122, 66.07000732],
       [ 1.33000004, 42.74000168, 66.99000549]])

In [15]:
mdc.process_specific_group("153lA00", f"{MDCATH_DIR}/mdcath_dataset_153lA00.h5",TEMP,REPL, 0)

(array([7, 1, 6, ..., 1, 6, 8]),
 array([[14.85000038, 53.81000137, 75.79000854],
        [15.06000042, 53.09000015, 76.45000458],
        [15.81999969, 54.06999969, 74.77999878],
        ...,
        [ 9.38000011, 41.35000229, 63.82000351],
        [ 8.82999992, 45.74000549, 69.07000732],
        [ 9.97000027, 45.99000549, 69.41000366]]),
 array([[ -4.61763048,  16.82621193,  -1.99926054],
        [ -0.73253453,  -6.33592844,   2.88446188],
        [-17.23141098,  28.47986221,  14.29771614],
        ...,
        [ -4.86854553,   1.57947326,  -0.28790951],
        [ -7.37175894,  -5.15880013,  -3.44312906],
        [ 12.22301483,   6.11317348,   5.47645569]]))

In [25]:
pdb_files = list(mdc.processed.keys())

In [23]:
len(pdb_files)

5184

In [29]:
pdb_files[0]

'153lA00'

In [43]:
f = h5py.File(f"{MDCATH_DIR}/mdcath_dataset_{pdb_files[0]}.h5", "r")

In [46]:
k = list(f.keys())

In [94]:
f[f"153lA00/{TEMP}/0/coords"][:]

array([[[ 1.48500004e+01,  5.38100014e+01,  7.57900085e+01],
        [ 1.50600004e+01,  5.30900002e+01,  7.64500046e+01],
        [ 1.58199997e+01,  5.40699997e+01,  7.47799988e+01],
        ...,
        [ 9.38000011e+00,  4.13500023e+01,  6.38200035e+01],
        [ 8.82999992e+00,  4.57400055e+01,  6.90700073e+01],
        [ 9.97000027e+00,  4.59900055e+01,  6.94100037e+01]],

       [[ 2.82000017e+00,  4.91399994e+01,  7.53900070e+01],
        [ 2.90000010e+00,  4.81800041e+01,  7.51300049e+01],
        [ 3.80000019e+00,  4.97299995e+01,  7.61900024e+01],
        ...,
        [ 2.69000030e+00,  4.03300018e+01,  6.02900009e+01],
        [ 6.70000017e-01,  4.32200012e+01,  6.60700073e+01],
        [ 1.33000004e+00,  4.27400017e+01,  6.69900055e+01]],

       [[ 5.66000032e+00,  5.18600044e+01,  7.87500076e+01],
        [ 4.71000004e+00,  5.19599991e+01,  7.90400009e+01],
        [ 5.85000038e+00,  5.06000061e+01,  7.80699997e+01],
        ...,
        [ 2.00000014e-02,  4.44900055e+01,

In [32]:
z, coords, forces = mdc.process_specific_group(pdb_files[0], f"{config.RAW_DATA_DIR}/mdcath/mdcath_dataset_{pdb_files[0]}.h5", "348", "0", 10)

In [33]:
coords.shape

(2855, 3)

In [39]:
mdc.idx[2]

('153lA00',
 '/mnt/home/ssledzieski/Projects/rocketshp/data/raw/mdcath/mdcath_dataset_153lA00.h5',
 '348',
 '0',
 2)

In [36]:
mdc.idx[0]

('153lA00',
 '/mnt/home/ssledzieski/Projects/rocketshp/data/raw/mdcath/mdcath_dataset_153lA00.h5',
 '348',
 '0',
 0)

In [34]:
mdc.get(0)

Data(z=[2855], pos=[2855, 3], neg_dy=[2855, 3], info='153lA00_348_0_0')

In [28]:
mdc.processed[pdb_files[0]][0]

('348', '0', 450)

In [18]:
vars(mdc).keys()

dict_keys(['url', 'source_file', 'root', 'numAtoms', 'numResidues', 'temperatures', 'skip_frames', 'pdb_list', 'min_gyration_radius', 'max_gyration_radius', 'alpha_beta_coil', 'numFrames', 'solid_ss', 'processed', 'num_conformers', 'idx', 'transform', 'pre_transform', 'pre_filter', 'log', '_indices', 'force_reload', 'total_size_mb'])